In [1]:
## Ran for eveything now with excluding the gauges that we don't want

# Create a set of dimensionless profiles
Read in all of the events, for all durations, for all gauges, for all ensemble members.  
Convert them to dimensionless profiles, with 12 values between 0 and 1.  
Each value is a dimensionless, cumulative rainfall value (cumulative rainfall at this timestep, normalised by the total event rainfall):
- 0 means no rainfall has occurred, and 
- 1 means the total event rainfall has been reached.  

If there are less than 12 values, then these are filled in with interpolation

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import pandas as pd
import re
from datetime import datetime

from Create_Profiles_Functions import *

quintile_mapping = {1: 'F2', 2: 'F1', 3: 'C', 4: 'B1', 5: 'B2'}

In [13]:
gauge_nums = range(0,1294)

# NIMROD - 30 mins
### Make profiles

In [4]:
# # # Initialize an empty DataFrame with the desired columns
# columns = [
#     'gauge_num',  'season', 'precip', 'Volume','Year', 'times', 'duration',
#     'normalized_rainfall', 'normalized_interpolated_rainfall_12', 'normalized_interpolated_rainfall_15',
#     'max_quintile_profile_12', 'max_quintile_profile_15', 'max_quintile_normalised_rain', 'max_quintile_raw_rain']
# df = pd.DataFrame(columns=columns)

# for nimrod_option in ["NIMROD_2.2km_filtered_100"]:
#     for gauge_num in range(0, 1293):
#         if gauge_num not in [444, 827, 888]:
#             print(f"gauge {gauge_num}")
#             files = [f for f in os.listdir(f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/NIMROD_30mins/{nimrod_option}/{gauge_num}/WholeYear") if f.endswith('.csv')]
#             files = np.sort(files)

#             for file in files:
#                 fp = f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/NIMROD_30mins/{nimrod_option}/{gauge_num}/WholeYear/{file}"
#                 if '2080' in fp:
#                     continue

#                 this_event = read_event(gauge_num, fp)
#                 trimmed_event = remove_leading_and_trailing_zeroes(this_event)
#                 real_trimmed_event, problem_events = remove_events_with_problems(trimmed_event, verbose=False)

#                 if real_trimmed_event is not None:
                    
#                     precip = real_trimmed_event['precipitation (mm/hr)']
#                     normalized_rainfall = create_normalised_event(real_trimmed_event['precipitation (mm/hr)'])
#                     cumulative_normalized_rainfall = create_cumulative_event(normalized_rainfall)
#                     interpolated15_cumulative_normalized_rainfall = interpolate_rainfall(cumulative_normalized_rainfall,15)
#                     interpolated12_cumulative_normalized_rainfall = interpolate_rainfall(cumulative_normalized_rainfall,12)
#                     interpolated15_incremental_normalized_rainfall = create_incremental_event(interpolated15_cumulative_normalized_rainfall)
#                     interpolated12_incremental_normalized_rainfall = create_incremental_event(interpolated12_cumulative_normalized_rainfall)

#                     max_quintile_profile_12 = find_part_with_most_rain(interpolated12_incremental_normalized_rainfall, 5)
#                     max_quintile_profile_15 = find_part_with_most_rain(interpolated15_incremental_normalized_rainfall, 5)
#                     max_quintile_normalised_rain = find_part_with_most_rain(normalized_rainfall, 5)
#                     max_quintile_raw_rain = find_part_with_most_rain(precip, 5)
                    
#                     duration = len(real_trimmed_event) / 2
#                     times = trimmed_event['times']
#                     season = get_season(trimmed_event['times'][0])
#                     year=extract_year(trimmed_event)
                    
#                 else:
#                     precip=trimmed_event['precipitation (mm/hr)']
#                     normalized_rainfall = None
#                     normalized_interpolated_rainfall_12 = None
#                     normalized_interpolated_rainfall_15 = None
#                     max_quintile_profile_12 = None
#                     max_quintile_profile_15 = None
#                     max_quintile_normalised_rain = None
#                     max_quintile_raw_rain = None
#                     duration = None
#                     season = None
#                     year=None
#                     times=None
                    
#                 # Append the row to the DataFrame
#                 df = df.append({
#                     'gauge_num': gauge_num,
#                     'season': season,
#                     'precip': precip.values,
#                     'Volume': sum(precip),
#                     'Year':year,
#                     'times':times, 
#                     'duration': duration,
#                     'normalized_rainfall': normalized_rainfall,
#                     'normalized_interpolated_rainfall_12': interpolated12_incremental_normalized_rainfall,
#                     'normalized_interpolated_rainfall_15': interpolated15_cumulative_normalized_rainfall,
#                     'max_quintile_profile_12': max_quintile_profile_12,
#                     'max_quintile_profile_15': max_quintile_profile_15,
#                     'max_quintile_normalised_rain': max_quintile_normalised_rain,
#                     'max_quintile_raw_rain': max_quintile_raw_rain
#                 }, ignore_index=True)
                
# df['Loading_profile12'] = df['max_quintile_profile_12'].map(quintile_mapping)
# df['Loading_profile15'] = df['max_quintile_profile_15'].map(quintile_mapping)
# df['Loading_profile_normalised_rain'] = df['max_quintile_normalised_rain'].map(quintile_mapping)
# df['Loading_profile_raw_rain'] = df['max_quintile_raw_rain'].map(quintile_mapping)

# with open(f"/nfs/a319/gy17m2a/PhD/ProcessedData/Profiles/NIMROD_30mins/WholeYear/{nimrod_option}_profiles_df.pkl", 'wb') as file:
#     pickle.dump(df, file)

# NIMROD 5 mins

# UKCP18
### Make and pickle profiles

In [14]:
# # Initialize an empty DataFrame with the desired columns
# Initialize an empty DataFrame with the desired columns
columns = [
    'gauge_num',  'season', 'precip', 'Volume','Year', 'times', 'duration',
    'normalized_rainfall', 'cumulative_normalized_rainfall', 'cumulative_rainfall',
    'interpolated15_cumulative_rainfall', 'interpolated12_cumulative_rainfall','interpolated15_incremental_rainfall','interpolated12_incremental_rainfall',
    'interpolated15_cumulative_normalized_rainfall','interpolated12_cumulative_normalized_rainfall','interpolated15_incremental_normalized_rainfall','interpolated12_incremental_normalized_rainfall',
    'max_quintile_raw_rain','max_quintile_profile_12', 'max_quintile_profile_15', 'max_quintile_normalised_rain', 'max_quintile_profile_12_norm', "max_quintile_profile_15_norm"]
df2 = pd.DataFrame(columns=columns)

for em in ["bb198"]:
    for gauge_num in range(0, 1293):
        if gauge_num not in [444, 827, 888]:
            print(f"gauge {gauge_num}")
            files = [f for f in os.listdir(f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/UKCP18_30mins/{em}/{gauge_num}/WholeYear/") if f.endswith('.csv')]
            files = np.sort(files)

            for file in files:
                fp = f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/UKCP18_30mins/{em}/{gauge_num}/WholeYear/{file}"
                if '2080' in fp:
                    continue

                this_event = read_event(gauge_num, fp)
                trimmed_event = remove_leading_and_trailing_zeroes(this_event)
                real_trimmed_event, problem_events = remove_events_with_problems(trimmed_event, verbose=False)

                if real_trimmed_event is not None:
                    
                    precip = real_trimmed_event['precipitation (mm/hr)']
                    normalized_rainfall = create_normalised_event(real_trimmed_event['precipitation (mm/hr)'])
                    cumulative_normalized_rainfall = create_cumulative_event(normalized_rainfall)
                    cumulative_rainfall = create_cumulative_event(precip)
                    
                    interpolated15_cumulative_rainfall = interpolate_rainfall(cumulative_rainfall,15)
                    interpolated12_cumulative_rainfall = interpolate_rainfall(cumulative_rainfall,12)
                    interpolated15_incremental_rainfall = create_incremental_event(interpolated15_cumulative_rainfall)
                    interpolated12_incremental_rainfall = create_incremental_event(interpolated12_cumulative_rainfall)
                    
                    interpolated15_cumulative_normalized_rainfall = interpolate_rainfall(cumulative_normalized_rainfall,15)
                    interpolated12_cumulative_normalized_rainfall = interpolate_rainfall(cumulative_normalized_rainfall,12)
                    interpolated15_incremental_normalized_rainfall = create_incremental_event(interpolated15_cumulative_normalized_rainfall)
                    interpolated12_incremental_normalized_rainfall = create_incremental_event(interpolated12_cumulative_normalized_rainfall)
                    
                    max_quintile_raw_rain = find_part_with_most_rain(precip, 5)
                    max_quintile_profile_12 = find_part_with_most_rain(interpolated12_incremental_rainfall, 5)
                    max_quintile_profile_15 = find_part_with_most_rain(interpolated15_incremental_rainfall, 5)
                    
                    max_quintile_normalised_rain = find_part_with_most_rain(normalized_rainfall, 5)
                    max_quintile_profile_12_norm = find_part_with_most_rain(interpolated12_incremental_normalized_rainfall, 5)
                    max_quintile_profile_15_norm = find_part_with_most_rain(interpolated15_incremental_normalized_rainfall, 5)
                    
                    duration = len(real_trimmed_event) / 2
                    times = trimmed_event['times'].values
                    season = get_season(trimmed_event['times'][0])
                    year=extract_year(trimmed_event)
                    
                else:
                    precip=trimmed_event['precipitation (mm/hr)']
                    normalized_rainfall = None
                    cumulative_normalized_rainfall = None
                    cumulative_rainfall = None
                    
                    interpolated15_cumulative_rainfall = None
                    interpolated12_cumulative_rainfall = None
                    interpolated15_incremental_rainfall = None
                    interpolated12_incremental_rainfall = None
                    
                    interpolated15_cumulative_normalized_rainfall = None
                    interpolated12_cumulative_normalized_rainfall = None
                    interpolated15_incremental_normalized_rainfall = None
                    interpolated12_incremental_normalized_rainfall = None
                    
                    max_quintile_raw_rain = None
                    max_quintile_profile_12 = None
                    max_quintile_profile_15 = None
                    
                    max_quintile_normalised_rain = None
                    max_quintile_profile_12_norm = None
                    max_quintile_profile_15_norm = None
                    
                    duration = None
                    season = None
                    year=None
                    times=None
                    
                # Append the row to the DataFrame
                df2 = df2.append({
                    'gauge_num': gauge_num,
                    'season': season,
                    'precip': precip.values,
                    
                    'Volume': sum(precip),
                    'Year':year,
                    'times':times, 
                    'duration': duration,
                    
                    'normalized_rainfall':normalized_rainfall,
                    'cumulative_normalized_rainfall':cumulative_normalized_rainfall,
                    'cumulative_rainfall':cumulative_rainfall,
                    
                    'interpolated15_cumulative_rainfall':interpolated15_cumulative_rainfall,
                    'interpolated12_cumulative_rainfall':interpolated12_cumulative_rainfall,                    
                    'interpolated15_incremental_rainfall':interpolated15_incremental_rainfall,
                    'interpolated12_incremental_rainfall': interpolated12_incremental_rainfall,
                    
                    'interpolated15_cumulative_normalized_rainfall':interpolated15_cumulative_normalized_rainfall,
                    "interpolated12_cumulative_normalized_rainfall":interpolated12_cumulative_normalized_rainfall,
                    'interpolated15_incremental_normalized_rainfall':interpolated15_incremental_normalized_rainfall,
                    'interpolated12_incremental_normalized_rainfall':interpolated12_incremental_normalized_rainfall,
                    
                    'max_quintile_raw_rain': max_quintile_raw_rain,
                    'max_quintile_profile_12': max_quintile_profile_12,
                    'max_quintile_profile_15': max_quintile_profile_15,
                    'max_quintile_normalised_rain': max_quintile_normalised_rain,
                    'max_quintile_profile_12_norm': max_quintile_profile_12_norm,
                    'max_quintile_profile_15_norm': max_quintile_profile_15_norm                    
                }, ignore_index=True)
                

df2['Loading_profile_raw_rain'] = df2['max_quintile_raw_rain'].map(quintile_mapping)
df2['Loading_profile12'] = df2['max_quintile_profile_12'].map(quintile_mapping)
df2['Loading_profile15'] = df2['max_quintile_profile_15'].map(quintile_mapping)
df2['Loading_profile_normalised_rain'] = df2['max_quintile_normalised_rain'].map(quintile_mapping)
df2['Loading_profile_norm_12'] = df2['max_quintile_profile_12_norm'].map(quintile_mapping)
df2['Loading_profile_norm_15'] = df2['max_quintile_profile_15_norm'].map(quintile_mapping)

df2 = add_duration_cats_based_on_data(df2)
df2 = add_duration_cats_predetermined(df2)

# with open(f"/nfs/a319/gy17m2a/PhD/ProcessedData/Profiles/UKCP18_30mins/{em}/profiles_df.pkl", 'wb') as file:
#     pickle.dump(df2, file)

gauge 0
gauge 1
gauge 2
gauge 3
gauge 4
gauge 5
gauge 6
gauge 7
gauge 8
gauge 9
gauge 10
Some dates were invalid and have been coerced to NaT:
gauge 11
gauge 12
gauge 13
Some dates were invalid and have been coerced to NaT:
Some dates were invalid and have been coerced to NaT:
gauge 14
gauge 15
gauge 16
gauge 17
gauge 18
gauge 19
gauge 20
gauge 21
gauge 22
gauge 23
gauge 24
gauge 25
Some dates were invalid and have been coerced to NaT:
gauge 26
gauge 27
gauge 28
Some dates were invalid and have been coerced to NaT:
gauge 29
Some dates were invalid and have been coerced to NaT:
gauge 30
gauge 31
gauge 32
gauge 33
Some dates were invalid and have been coerced to NaT:
gauge 34
gauge 35
gauge 36
gauge 37
Some dates were invalid and have been coerced to NaT:
gauge 38
gauge 39
gauge 40
gauge 41
gauge 42
gauge 43
gauge 44
gauge 45
gauge 46
gauge 47
gauge 48
gauge 49
gauge 50
gauge 51
gauge 52
gauge 53
gauge 54
gauge 55
gauge 56
gauge 57
Some dates were invalid and have been coerced to NaT:
ga

gauge 359
gauge 360
Some dates were invalid and have been coerced to NaT:
gauge 361
Some dates were invalid and have been coerced to NaT:
gauge 362
gauge 363
gauge 364
gauge 365
gauge 366
gauge 367
gauge 368
gauge 369
gauge 370
Some dates were invalid and have been coerced to NaT:
gauge 371
gauge 372
Some dates were invalid and have been coerced to NaT:
gauge 373
gauge 374
gauge 375
gauge 376
gauge 377
Some dates were invalid and have been coerced to NaT:
Some dates were invalid and have been coerced to NaT:
gauge 378
gauge 379
gauge 380
gauge 381
gauge 382
gauge 383
gauge 384
gauge 385
gauge 386
gauge 387
gauge 388
gauge 389
gauge 390
gauge 391
gauge 392
Some dates were invalid and have been coerced to NaT:
gauge 393
Some dates were invalid and have been coerced to NaT:
Some dates were invalid and have been coerced to NaT:
Some dates were invalid and have been coerced to NaT:
gauge 394
Some dates were invalid and have been coerced to NaT:
gauge 395
Some dates were invalid and have bee

gauge 694
gauge 695
gauge 696
gauge 697
gauge 698
gauge 699
Some dates were invalid and have been coerced to NaT:
gauge 700
Some dates were invalid and have been coerced to NaT:
Some dates were invalid and have been coerced to NaT:
gauge 701
gauge 702
Some dates were invalid and have been coerced to NaT:
gauge 703
Some dates were invalid and have been coerced to NaT:
gauge 704
gauge 705
gauge 706
gauge 707
gauge 708
gauge 709
gauge 710
gauge 711
gauge 712
gauge 713
gauge 714
gauge 715
gauge 716
gauge 717
gauge 718
gauge 719
gauge 720
gauge 721
gauge 722
gauge 723
gauge 724
gauge 725
gauge 726
gauge 727
gauge 728
Some dates were invalid and have been coerced to NaT:
gauge 729
gauge 730
gauge 731
gauge 732
gauge 733
gauge 734
gauge 735
gauge 736
gauge 737
gauge 738
gauge 739
gauge 740
gauge 741
gauge 742
Some dates were invalid and have been coerced to NaT:
gauge 743
gauge 744
gauge 745
gauge 746
gauge 747
gauge 748
gauge 749
gauge 750
gauge 751
gauge 752
gauge 753
gauge 754
gauge 755
ga

Some dates were invalid and have been coerced to NaT:
Some dates were invalid and have been coerced to NaT:
gauge 1013
Some dates were invalid and have been coerced to NaT:
Some dates were invalid and have been coerced to NaT:
gauge 1014
Some dates were invalid and have been coerced to NaT:
gauge 1015
Some dates were invalid and have been coerced to NaT:
Some dates were invalid and have been coerced to NaT:
gauge 1016
Some dates were invalid and have been coerced to NaT:
Some dates were invalid and have been coerced to NaT:
gauge 1017
Some dates were invalid and have been coerced to NaT:
Some dates were invalid and have been coerced to NaT:
gauge 1018
Some dates were invalid and have been coerced to NaT:
Some dates were invalid and have been coerced to NaT:
gauge 1019
Some dates were invalid and have been coerced to NaT:
Some dates were invalid and have been coerced to NaT:
gauge 1020
Some dates were invalid and have been coerced to NaT:
gauge 1021
Some dates were invalid and have been

gauge 1233
gauge 1234
gauge 1235
gauge 1236
Some dates were invalid and have been coerced to NaT:
Some dates were invalid and have been coerced to NaT:
gauge 1237
gauge 1238
gauge 1239
Some dates were invalid and have been coerced to NaT:
gauge 1240
gauge 1241
gauge 1242
gauge 1243
gauge 1244
gauge 1245
gauge 1246
gauge 1247
Some dates were invalid and have been coerced to NaT:
gauge 1248
gauge 1249
gauge 1250
gauge 1251
gauge 1252
gauge 1253
gauge 1254
gauge 1255
gauge 1256
gauge 1257
Some dates were invalid and have been coerced to NaT:
gauge 1258
Some dates were invalid and have been coerced to NaT:
gauge 1259
gauge 1260
gauge 1261
gauge 1262
gauge 1263
gauge 1264
Some dates were invalid and have been coerced to NaT:
Some dates were invalid and have been coerced to NaT:
gauge 1265
gauge 1266
gauge 1267
Some dates were invalid and have been coerced to NaT:
Some dates were invalid and have been coerced to NaT:
gauge 1268
Some dates were invalid and have been coerced to NaT:
Some dates

In [15]:
with open(f"/nfs/a319/gy17m2a/PhD/ProcessedData/Profiles/UKCP18_30mins/{em}/profiles_df.pkl", 'wb') as file:
    pickle.dump(df2, file)